In [1]:
import pandas as pd
import numpy as np

import os

def load_hotel_reserve():
  customer_tb = pd.read_csv('../awesomebook-master/data/customer.csv')
  hotel_tb = pd.read_csv('../awesomebook-master/data/hotel.csv')
  reserve_tb = pd.read_csv('../awesomebook-master/data/reserve.csv')
  return customer_tb, hotel_tb, reserve_tb


def load_holiday_mst():
  holiday_tb = pd.read_csv('../awesomebook-master/data/holiday_mst.csv',
                           index_col=False)
  return holiday_tb


def load_production():
  production_tb = pd.read_csv('../awesomebook-master/data/production.csv')
  return production_tb


def load_production_missing_num():
  production_tb = pd.read_csv('../awesomebook-master/data/production_missing_num.csv')
  return production_tb


def load_production_missing_category():
  production_tb = pd.read_csv('../awesomebook-master/data/production_missing_category.csv')
  return production_tb


def load_monthly_index():
  monthly_index_tb = \
    pd.read_csv('../awesomebook-master/data/monthly_index.csv')
  return monthly_index_tb


def load_meros_txt():
  with open('../awesomebook-master/data/txt/meros.txt', 'r') as f:
    meros = f.read()
    f.close()
  return meros


In [2]:
customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()

In [3]:
# sex가 man일 때 TRUE 값을 가지는 불린 형을 추가
# 이 코드는 as.type 함수가 아니여도 불린형으로 변환된다
customer_tb[['sex_is_man']] = (customer_tb[['sex']] == 'man').astype('bool')

# sex를 카테고리 형으로 변환
customer_tb['sex_c'] = \
  pd.Categorical(customer_tb['sex'], categories=['man', 'woman'])

# astype 함수로도 변환 가능
# customer_tb['sex_c'] = customer_tb['sex_c'].astype('category')

# 인덱스 데이터는 codes에 저장된다
customer_tb['sex_c'].cat.codes

# 마스터 데이터는 categories에 저장된다
customer_tb['sex_c'].cat.categories


Index(['man', 'woman'], dtype='object')

In [4]:
# 더미 변수로 만들기 전에 카테고리 형으로 변환
customer_tb['sex'] = pd.Categorical(customer_tb['sex'])

# get_dummies 함수로 sex를 더미 변수화
# drop_first를 False로 하면, 카테고리값의 모든 종류 값의 더미 플래그를 생성
dummy_vars = pd.get_dummies(customer_tb['sex'], drop_first=False)

dummy_vars

,man,woman
0,True,False
1,True,False
2,False,True
3,True,False
4,True,False
...,...,...
995,True,False
996,True,False
997,False,True
998,False,True


In [5]:
# pd.Categorical로 category형으로 변환
customer_tb['age_rank'] = \
  pd.Categorical(np.floor(customer_tb['age']/10)*10)

# 마스터 데이터에 '60 이상'을 추가
#customer_tb['age_rank'].cat.add_categories(['60 이상'], inplace=True)
customer_tb['age_rank'] = customer_tb['age_rank'].cat.add_categories(['60 이상'])


# 집약할 데이터 변경하기
# category형은 = 또는 != 만 판정할 수 있기 때문에 isin 함수를 이용
customer_tb.loc[customer_tb['age_rank'] \
           .isin([60.0, 70.0, 80.0]), 'age_rank'] = '60 이상'

# 사용되지 않는 마스터 데이터를 제거
#customer_tb['age_rank'].cat.remove_unused_categories(inplace=True)
customer_tb['age_rank'] = customer_tb['age_rank'].cat.remove_unused_categories()


In [ ]:

customer_tb['sex_and_age'] = pd.Categorical(
  # 연결할 열을 추출
  customer_tb[['sex', 'age']]

    # lambda 함수에서 sex와 10살 단위로 구분한 age 사이에 _를 추가하여 연결
    .apply(lambda x: '{}_{}'.format(x[0], np.floor(x[1] / 10) * 10),
           axis=1)
)
customer_tb['sex_and_age']

/tmp/ipykernel_15953/4080997925.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  .apply(lambda x: '{}_{}'.format(x[0], np.floor(x[1] / 10) * 10),


0        man_40.0
1        man_30.0
2      woman_40.0
3        man_40.0
4        man_30.0
          ...    
995      man_40.0
996      man_30.0
997    woman_30.0
998    woman_40.0
999      man_30.0
Name: sex_and_age, Length: 1000, dtype: category
Categories (14, object): ['man_20.0', 'man_30.0', 'man_40.0', 'man_50.0', ..., 'woman_50.0', 'woman_60.0', 'woman_70.0', 'woman_80.0']

In [10]:

production = load_production()

# 제품 종류별 불량률
fault_cnt_per_type = production \
  .query('fault_flg') \
  .groupby('type')['fault_flg'] \
  .count()

# 제품 종류별 제조수
type_cnt = production.groupby('type')['fault_flg'].count()

production['type_fault_rate'] = production[['type', 'fault_flg']] \
  .apply(lambda x:
         (fault_cnt_per_type[x[0]] - int(x[1])) / (type_cnt[x[0]] - 1),
         axis=1)


/tmp/ipykernel_15953/233461485.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  (fault_cnt_per_type[x[0]] - int(x[1])) / (type_cnt[x[0]] - 1),


In [ ]:
production_missc_tb = load_production_missing_category()

# KNeighborsClassifier를 sklearn 라이브러이에서 읽어 들임
from sklearn.neighbors import KNeighborsClassifier

# replace 함수로 None을 nan으로 변환
production_missc_tb.replace('None', np.nan, inplace=True)

# 결손이 발생하지 않은 데이터를 추출
train = production_missc_tb.dropna(subset=['type'], inplace=False)

# 결손이 발생한 데이터를 추출
test = production_missc_tb \
  .loc[production_missc_tb.index.difference(train.index), :]

# knn 모델 생성, n_neighbors는 knn의 k 파라미터
kn = KNeighborsClassifier(n_neighbors=3)

# knn 모델 학습
kn.fit(train[['length', 'thickness']], train['type'])

# knn 모델로 예측값을 계산하여 type을 보완
test['type'] = kn.predict(test[['length', 'thickness']])
